In [ ]:
!pip install botorch

In [ ]:
import os
import json
import numpy as np
import torch
import gpytorch
from botorch.models.transforms import Normalize
from botorch.fit import fit_gpytorch_mll
from gpytorch.priors import GammaPrior
from gpytorch.kernels import ScaleKernel,RBFKernel,AdditiveKernel,MaternKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from  botorch.acquisition.multi_objective.logei import qLogExpectedHypervolumeImprovement
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.sampling.normal import SobolQMCNormalSampler
import matplotlib.pyplot as plt
import seaborn as sns

## ===============================GP Training=====================================

In [ ]:
# Load data
json_file = "pism_rmse.json"
with open(json_file, "r") as f:
    log_data = json.load(f)

# Extract Params
q = [entry["q"] for entry in log_data]
pmin = [entry["pmin"] for entry in log_data]
pmax = [entry["pmax"] for entry in log_data]
zmin = [entry["zmin"] for entry in log_data]
zmax = [entry["zmax"] for entry in log_data]
lapse = [entry["lapse"] for entry in log_data]
essa = [entry["essa"] for entry in log_data]
esia = [entry["esia"] for entry in log_data]

# Extract Target
rmse_thk = [entry["thkRMSE"] for entry in log_data]
rmse_vel = [entry["velmagRMSE"] for entry in log_data]

# Transform Parameters and Target
rmse_thk_r = torch.tensor(rmse_thk,dtype=torch.double).view(-1, 1)  # Reshape to (n, 1)
rmse_vel_r = torch.tensor(rmse_vel,dtype=torch.double).view(-1, 1) 

# Combine parameters into a list of lists for processing
params = list(zip(q, pmin, pmax, zmin, zmax, lapse, essa, esia))
params = torch.tensor([list(p) for p in params], dtype=torch.double) 

# Set params and rmse for multi-objective GP
params_multi=torch.cat([params,params])
rmse_mult=torch.cat([-rmse_thk_r,-rmse_vel_r],dim=1)

In [ ]:
#The range of parameters (q, phi_min, phi_max, z_min, z_max, gamma_M, e_ssa, e_sia)
bounds_parameters = torch.tensor([
            [0,1],
            [0,15],
            [35,46],
            [-1000,0],
            [0,1000],
            [0,5],
            [0.1,2],
            [1,5],
        ],dtype=torch.double).T

def initialize_model(train_x, train_obj):
    bounds = bounds_parameters
    models = []
    #Ice Surface Elevation GP
    train_y = train_obj[..., 0:1]
    models.append(
        SimpleEGP(
            train_x, train_y
        )
    )

    #Ice Surface Velocity GP
    train_y = train_obj[..., 1:2]
    models.append(
        SimpleVGP(
            train_x, train_y
        )
    )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [ ]:
# Define Kernel
class SimpleEGP(SingleTaskGP):
    def __init__(self, train_X, train_Y):
        bound = bounds_parameters
        
        super().__init__(train_X, train_Y, input_transform=Normalize(d=8,bounds=bound), outcome_transform=Standardize(m=1))

        # Create RBF kernel for dims except phi_max and e_ssa
        rbf_dims = [0, 1, 3, 4, 5, 7]
        kernel_rbf = RBFKernel(ard_num_dims=len(rbf_dims), active_dims=torch.tensor(rbf_dims))
        
        # Create Matern kernel for phi_max and e_ssa
        matern_dims = [2,6]
        kernel_matern = MaternKernel(nu=1.5, ard_num_dims=len(matern_dims), active_dims=torch.tensor(matern_dims))
        
        # Register prior only on e_sia
        kernel_rbf.register_prior(
            "lengthscale_prior_last_dim", GammaPrior(2, 6),
            lambda m: m.lengthscale[..., -1]
        )

        # Combine kernels using additive kernel
        mixed_kernel = AdditiveKernel(kernel_rbf, kernel_matern)

        # Wrap in ScaleKernel
        self.covar_module = ScaleKernel(mixed_kernel)

# Kernel for velocity
class SimpleVGP(SingleTaskGP):
    def __init__(self, train_X, train_Y):
        bound = bounds_parameters
        
        super().__init__(train_X, train_Y, input_transform=Normalize(d=8,bounds=bound), outcome_transform=Standardize(m=1))

        # Create RBF kernel for e_sia
        rbf_dims = [7]
        kernel_rbf = RBFKernel(ard_num_dims=len(rbf_dims), active_dims=torch.tensor(rbf_dims))

        # Create Matern kernel for other dims
        matern2_dims = [0, 1, 3, 4, 5]
        kernel_matern2 = MaternKernel(nu=2.5, ard_num_dims=len(matern2_dims), active_dims=torch.tensor(matern2_dims))
        
        # Create Matern kernel for phi_max and e_ssa
        matern_dims = [2,6]
        kernel_matern = MaternKernel(nu=1.5, ard_num_dims=len(matern_dims), active_dims=torch.tensor(matern_dims))

        # Combine kernels using additive kernel
        mixed_kernel = AdditiveKernel(kernel_rbf, kernel_matern2, kernel_matern)

        # Wrap in ScaleKernel
        self.covar_module = ScaleKernel(mixed_kernel)

d=8

In [ ]:
# Train GP
mll, model = initialize_model(params, rmse_mult)
fit_gpytorch_mll(mll)

In [ ]:
# Obtain the model output on training data
a=model.train_inputs[0]
model_output = model(a[0],a[0])

# Compute the marginal log-likelihood
mll_value = mll(model_output, model.train_targets).item()

print("Marginal Log-Likelihood:", mll_value)

In [ ]:
# Print the lengthscale for ice surface elevation
print(model.models[0].covar_module.base_kernel.kernels[0].lengthscale.detach())
print(model.models[0].covar_module.base_kernel.kernels[1].lengthscale.detach())

# Print the lengthscale for ice surface velocity magnitude
print(model.models[1].covar_module.base_kernel.kernels[0].lengthscale.detach())
print(model.models[1].covar_module.base_kernel.kernels[1].lengthscale.detach())
print(model.models[1].covar_module.base_kernel.kernels[2].lengthscale.detach())

In [ ]:
# Candidates size
BATCH_SIZE = 4
NUM_RESTARTS = 15
RAW_SAMPLES = 512

standard_bounds = torch.zeros(2, 7)
standard_bounds[1] = 1
standard_bounds = bounds_parameters
ref_point=torch.tensor([-360,-360],dtype=torch.float64)

def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(train_x).mean
    partitioning = FastNondominatedPartitioning(
        ref_point=ref_point,
        Y=pred,
    )
    acq_func = qLogExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    
    # observe new values
    new_x = candidates
    return new_x

In [ ]:
# Define the qehvi acquisition modules using a QMC sampler
qehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))

In [ ]:
# Compute hypervolume
bd = DominatedPartitioning(ref_point=ref_point, Y=rmse_mult)
volume = bd.compute_hypervolume().item()
print(volume)

In [ ]:
# Next generation candidate
candidate = optimize_qehvi_and_get_observation(model, params, rmse_mult, qehvi_sampler)

In [ ]:
# Print the predicted RMSE of the candidate
print(-model.posterior(candidate).mean)
print(model.posterior(candidate).variance)

In [ ]:
# Print the Candidate
print(candidate)

## ======================= Bayesian Calibration ============================

In [ ]:
def transform_model(mean,variance,option):
    # Normalize & Standarize
    mean_cal = mean[0,option].detach().numpy()
    variance_cal = variance[0,option].detach().numpy()
    return [(mean_cal-transformmean[option])/transformstd[option],variance_cal/transformstd[option]**2]

def logll(x_torch,n):
    # Calculate the log-likelihood

    # The posterior in GP
    posterior = model.posterior(x_torch)
    mean, variance = posterior.mean, posterior.variance
    
    # Ice surface elevation
    mean_cal,variance_cal=transform_model(mean,variance,0)
    likelihood1 = -0.5*np.log(2*np.pi*variance_cal)-mean_cal**2/(2*variance_cal)
    
    # Ice surface velocity magnitude
    mean_cal,variance_cal = transform_model(mean,variance,1)
    likelihood2 = -0.5*np.log(2*np.pi*variance_cal)-mean_cal**2/(2*variance_cal)

    # Return weighted mean
    return n*likelihood1+(1-n)*likelihood2 

In [ ]:
# Get the transform in GP
transform1=model.models[0].input_transform
transformmean=[model.models[0].outcome_transform.means.detach().numpy()[0,0],model.models[1].outcome_transform.means.detach().numpy()[0,0]]
transformstd=[model.models[0].outcome_transform.stdvs.detach().numpy()[0,0],model.models[1].outcome_transform.stdvs.detach().numpy()[0,0]]

# Start Point
x=np.array([[1.0000,6.4074,46.0000,-362.5400,0.0000,2.5183,1.1229,1.5745]])
torch_x=torch.tensor(x, dtype=torch.double)
posterior = model.posterior(torch_x)  # Predict for candidates
mean_start, variance_mean_start = posterior.mean, posterior.variance

In [ ]:
# MCMC

n = 100000 # Number
x_examples = np.random.rand(n+1, 8) # Store
scale = 0.2 # Neiborhood
x_examples[0,:] = x

# Control the weight of log-likelihood (0-velocity calibration, 1-elevation calibration)
weight = 1

for i in range(n):
    x_old = transform1(torch_x).detach().numpy()
    
    k = np.array([np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale),np.random.normal(loc=0.0, scale=scale)])
    x_new = x_old+k

    # Reflection
    x_ref = np.where(x_new[0]<0,-x_new[0],x_new[0])
    x_ref = np.where(x_new[0]>1,2-x_new[0],x_new[0])
    
    # Avoid little outside bound
    x_new[0] = np.clip(x_ref,0,1)
    
    # Data Pre-process
    x_old_c = transform1.untransform(torch.tensor(x_old, dtype=torch.double))
    x_new_c = transform1.untransform(torch.tensor(x_new, dtype=torch.double))
    
    # Metropolis Rule
    
    p = np.random.rand(1)
    
    # Accept
    if logll(x_torch=x_new_c,n=weight)>logll(x_torch=x_old_c,n=weight):
        x_old = x_new
        x_old_store = x_new_c.detach().numpy()
    elif np.exp(logll(x_torch=x_new_c,n=weight)-logll(x_torch=x_old_c,n=weight))<p:
        x_old = x_new
        x_old_store = x_new_c.detach().numpy()
    # Reject
    else:
        x_old = x_old
        x_old_store = x_old_c.detach().numpy()

    # Store
    x_examples[i+1,:] = x_old_store

    if np.mod(i,10000)==0:
        print(i)

In [ ]:
# Define the number of dimensions to plot
num_dims = 8
bins = 50
title = ["q","phi_min","phi_max","z_min","z_max","gamma_M","e_ssa","e_sia"]

# Create subplots
fig, axes = plt.subplots(2, 4, figsize=(14, 6))
axes = axes.flatten()
xlims = [(0, 1), (0, 15), (35, 46), (-1000, 0), (0, 1000), (0, 5), (0, 2), (1, 5)]

for dim_index in range(num_dims):
    values = x_examples[:, dim_index]
    
    # Plot histogram
    sns.histplot(values, bins=bins, kde=True, alpha=0.5, stat='probability',ax=axes[dim_index],color='dimgray')
    
    # Set title for each subplot
    axes[dim_index].set_title(f'{title[dim_index]}')
    
# Show and Save
plt.tight_layout()
#plt.savefig('count_both.png')
plt.show()